# SVM Classifier Training Model

In [7]:
import cv2
import os
import numpy as np
from skimage.feature import hog 
from sklearn import svm
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score , classification_report
import joblib

In [8]:
DATASET_PATH = r'C:\Users\Welcome\Downloads\data\data\test' # DATASET 
IMG_SIZE = 128

X =[]
y = []

# Loop through categories

for label, category in enumerate(["cats" ,"dogs"]):
    path = os.path.join(DATASET_PATH , category)

    for img_name in os.listdir(path):
        img_path = os.path.join(path , img_name)

        img = cv2.imread(img_path)
        if img is None:
            continue


        gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
        gray = cv2.resize(gray, (IMG_SIZE, IMG_SIZE))


        features = hog(gray, orientations=9, pixels_per_cell=(8,8), cells_per_block=(2,2), visualize=False)

        X.append(features)
        y.append(label)

X = np.array(X)
y = np.array(y)



# Train test split

In [10]:
X_train , X_test , y_train , y_test = train_test_split(X, y , test_size=0.2, random_state=42)

# train_SVM

In [11]:
model = svm.SVC(kernel="linear" , probability=True)
model.fit(X_train, y_train)

SVC(kernel='linear', probability=True)

# Evaluation

In [13]:
y_pred = model.predict(X_test)
print("Accuracy:" , accuracy_score(y_test , y_pred))
print("classification_report:" , classification_report(y_test,y_pred))

Accuracy: 0.6197530864197531
classification_report:               precision    recall  f1-score   support

           0       0.65      0.58      0.61       209
           1       0.60      0.66      0.63       196

    accuracy                           0.62       405
   macro avg       0.62      0.62      0.62       405
weighted avg       0.62      0.62      0.62       405



# Save model

In [9]:
joblib.dump(model, "dog_cat_model.pkl")

print("Model Saved Successfully")

Model Saved Successfully


# Webcam classification script

In [10]:
import cv2
import joblib
import numpy as np
from skimage.feature import hog

In [ ]:
IMG_SIZE = 128

# Local trained model
model = joblib.load("dog_cat_model.pkl")

cap = cv2.VideoCapture(0)

while True:
    ret , frame = cap.read()
    if not ret:
        break

    # preprocess frame
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    resized = cv2.resize(gray, (IMG_SIZE,IMG_SIZE))

    features = hog(resized, orientations=9, pixels_per_cell=(8,8), cells_per_block=(2,2), visualize=False)

    prediction = model.predict([features])[0]

    label = "CAT" if prediction == 0 else "DOG"

    cv2.putText(frame, label, (20,50) , cv2.FONT_HERSHEY_SIMPLEX,1, (0,255,0),2)

    cv2.imshow("Pet Classifier" , frame)


    if cv2.waitKey(1) & 0xff == ord("q"):
        break

cap.release()
cv2.destroyAllWindows()